In [ ]:
!mkdir templates -p

In [ ]:
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>UTS Cafeteria GPT</title>
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css">
  <style>
    body {
        background-image:  url(https://lh5.googleusercontent.com/p/AF1QipPr5YfZjO-FRPh9Xqv83eJ19l6Kl3CdOsAJ6t4-=w141-h118-n-k-no-nu);
        background-color: rgba(0, 0, 0, 0.8);
        background-repeat: no-repeat;
        background-position: center;
        background-attachment: fixed;
        background-size: cover;
        background-blend-mode: darken;
      font-family: Arial, sans-serif;
    }

    .container {
      max-width: 600px;
      margin: 100px auto;
      background-color: #fff;
      border-radius: 8px;
      box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
      padding: 20px;
    }

    #chatContainer {
      height: 600px;
      overflow-y: auto;
      border: 1px solid #ccc;
      border-radius: 8px;
      padding: 10px;
      margin-bottom: 20px;
      background-color: #f9f9f9;
    }

    .messageContainer {
      margin-bottom: 10px;
      overflow: hidden;
    }

    .message {
      padding: 10px;
      border-radius: 8px;
      max-width: 80%;
      word-wrap: break-word;
    }

    .userMessage {
      background-color: #007bff;
      color: #fff;
      float: right;
    }

    .botMessage {
      background-color: #28a745;
      color: #fff;
      float: left;
    }

    .avatar {
      width: 30px;
      height: 30px;
      border-radius: 50%;
      margin-right: 10px;
      float: left;
    }

    .typingIndicator {
      display: inline-block;
      width: 10px;
      height: 10px;
      margin-right: 5px;
      background-color: #ccc;
      border-radius: 50%;
      animation: typingAnimation 1s infinite;
    }

    @keyframes typingAnimation {
      0% {
        background-color: #ccc;
      }
      50% {
        background-color: transparent;
      }
      100% {
        background-color: #ccc;
      }
    }

    /* Style for form input and button */
    .form-group {
      display: flex;
      align-items: center;
    }

    #question {
      flex: 1;
      margin-right: 10px;
      padding: 8px;
      border: 1px solid #ccc;
      border-radius: 5px;
    }

    #submitBtn {
      padding: 8px 20px;
      background-color: #007bff;
      color: #fff;
      border: none;
      border-radius: 5px;
      cursor: pointer;
    }
  </style>
</head>
<body>

<div class="container">
  <h1 class="text-center">Cafeteria GPT</h1>

  <div id="chatContainer"></div>

  <form id="chatForm" action="#" method="POST">
    <div class="form-group">
      <input type="text" id="question" name="question" placeholder="Type your message..." autocomplete="off">
      <button type="submit" id="submitBtn">Send</button>
    </div>
  </form>
</div>

<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.9.1/jquery.min.js"></script>
<script>
  $(document).ready(function() {
    // 定义一个基类
    function ChatMessage(content, isUser) {
      this.content = content;
      this.isUser = isUser;
    }

    // 添加一个方法到 ChatMessage 的原型上，用于展示消息
    ChatMessage.prototype.displayMessage = function() {
      var messageClass = this.isUser ? "userMessage" : "botMessage";
      var messageContent = "<div class='message " + messageClass + "'>" + this.content + "</div>";
      var avatar = this.isUser ? "" : "<img src='https://cdn-icons-png.flaticon.com/512/4712/4712109.png' class='avatar'>";
      return "<div class='messageContainer'>" + avatar + messageContent + "</div>";
    };

    // 派生一个具体类，继承自基类 ChatMessage
    function UserMessage(content) {
      ChatMessage.call(this, content, true);
    }

    // 使用原型链继承基类的方法
    UserMessage.prototype = Object.create(ChatMessage.prototype);
    UserMessage.prototype.constructor = UserMessage;

    // 派生一个具体类，继承自基类 ChatMessage
    function BotMessage(content) {
      ChatMessage.call(this, content, false);
    }

    // 使用原型链继承基类的方法
    BotMessage.prototype = Object.create(ChatMessage.prototype);
    BotMessage.prototype.constructor = BotMessage;

    // 创建一个聊天界面类，用于处理用户输入和显示消息
    function ChatInterface() {
      this.messages = [];
    }

    ChatInterface.prototype.sendMessage = function(message) {
      var userMessage = new UserMessage(message.trim());
      this.messages.push(userMessage);
      this.displayMessages();
      // 模拟机器人回复
      this.showTypingIndicator();
      var that = this;
      $.ajax({
        url: '/get_response', // 将此 URL 替换为后端路由，用于获取机器人的回复消息
        type: 'POST',
        data: { question: message },
        success: function(response) {
          that.botReply(response);
        },
        error: function(xhr, status, error) { //exception handling
          console.error("Error:", error);
          that.botReply("Sorry, Got Some Error, Please press f12 to check the error")
        }
      });
    };

    ChatInterface.prototype.botReply = function(response) {
      var botMessage = new BotMessage(response);
      // 移除等待特效
      this.messages.pop();
      this.messages.push(botMessage);
      this.displayMessages();
    };

    ChatInterface.prototype.showTypingIndicator = function() {
      var typingIndicator = "<span class='typingIndicator'></span><span class='typingIndicator'></span><span class='typingIndicator'></span>";
      var botMessage = new BotMessage(typingIndicator);
      this.messages.push(botMessage);
      this.displayMessages();
    };

    ChatInterface.prototype.displayMessages = function() {
      var chatContainer = $("#chatContainer");
      chatContainer.empty();
      this.messages.forEach(function(message) {
        chatContainer.append(message.displayMessage());
      });
      scrollChatContainer();
    };

    // 滚动聊天容器到底部
    function scrollChatContainer() {
      var chatContainer = $("#chatContainer");
      chatContainer.scrollTop(chatContainer.prop("scrollHeight"));
    }

    // 创建一个聊天界面实例
    var chatInterface = new ChatInterface();

    // 初始问候消息
    var initialGreeting = "Hello and Welcome to Cafeteria Chatbot! What can I do for you?";
    var botMessage = new BotMessage(initialGreeting);
    chatInterface.messages.push(botMessage);
    chatInterface.displayMessages();

    // 监听表单提交事件
    $("#chatForm").submit(function(event) {
      event.preventDefault();
      var userInput = $("#question").val();
      if (userInput) {
        chatInterface.sendMessage(userInput);
        $("#question").val("");
      }
    });

    // Handle constant responses for specific user inputs
    $("#chatForm").on("input", function() {
      var userInput = $("#question").val().toLowerCase();
      if (lowerCaseMessage.includes('hello') || lowerCaseMessage.includes('hi')) {
          return 'Hello! How can I assist you?';
      } else if (lowerCaseMessage.includes('help')) {
            return 'Sure, I can help you. What do you need assistance with?';
      } else if (lowerCaseMessage.includes('bye') || lowerCaseMessage.includes('goodbye')) {
            return 'Goodbye! Have a great day!';
      } else {
            return "I'm sorry, I didn't understand that. Can you please rephrase?";
      }
    });

  });
</script>

</body>
</html>




Overwriting templates/index.html


In [ ]:
import os
import threading

In [ ]:
!pip install pyngrok

In [ ]:
!killall ngrok

In [ ]:
from flask import Flask, render_template, request, jsonify
from pyngrok import ngrok
!ngrok version
os.environ["FLASK_DEBUG"] = "development"
# template_folder = '/gdrive/MyDrive/AIWeb/template'
app = Flask(__name__)
port = 5000
ngrok.set_auth_token("ngrok_token")
public_url = ngrok.connect(port).public_url
print("* ngrok tunnel \"{}\">\"http://127.0.0.1:{}\"".format(public_url, port))

app.config["BASE_URL"] = public_url
message = ""
question = ""
questionArray = []
chat_history = []
@app.route("/")
@app.route("/index", methods=['GET', 'POST'])
def index():
  global message
  global question
  # global questionArray
  if request.method == 'POST':
    # chat_history = [(query, result["answer"])]
    # question = request.form.get("question")
    getQuestion = request.form.get("question")
    result = chain({"question": getQuestion, "chat_history": chat_history})
    question = getQuestion
    message = result['answer']
    # questionArray.append()
    # print(result)
    # print(chat_history)
  return render_template('index.html', message=message, question=question)
@app.route('/get_response', methods=['POST'])
def get_response():
    user_question = request.form['question']
    result = chain({"question": user_question, "chat_history": chat_history})
    bot_response = result['answer']
    return bot_response
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

ngrok version 3.9.0
pyngrok version 7.1.6
* ngrok tunnel "https://1b00-34-125-132-136.ngrok-free.app">"http://127.0.0.1:5000"
 * Serving Flask app '__main__'


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from flask import Flask, render_template, request
from pyngrok import ngrok
!ngrok version
os.environ["FLASK_ENV"] = "development"
template_folder = '/gdrive/MyDrive/AIWeb/template'
app = Flask(__name__,template_folder=template_folder)
port = 5000
ngrok.set_auth_token("ngrok_token")
public_url = ngrok.connect(port).public_url
print("* ngrok tunnel \"{}\">\"http://127.0.0.1:{}\"".format(public_url, port))

app.config["BASE_URL"] = public_url

@app.route("/", methods=['GET', 'POST'])
@app.route("/index", methods=['GET', 'POST'])
def index():
  global message
  if request.method == 'POST':
    chat_history = [(question, result["answer"])]
    question = request.form.get("question")
    # query = request.form.get("question")
    result = chain({"question": question, "chat_history": chat_history})
    message = result['answer']
    print(result['answer'])
    # print(chat_history)
  return render_template('index.html', message=message)
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

ngrok version 3.9.0
pyngrok version 7.1.6
* ngrok tunnel "https://f935-34-125-132-136.ngrok-free.app">"http://127.0.0.1:5000"
